# Start

In [4]:
import spacy

In [5]:
!python -m spacy download ru_core_news_sm

     ---------------------------------------- 0.0/15.3 MB ? eta -:--:--
     -- ------------------------------------- 1.0/15.3 MB 10.1 MB/s eta 0:00:02
     ------------------------- -------------- 9.7/15.3 MB 33.6 MB/s eta 0:00:01
     ---------------------------------------- 15.3/15.3 MB 38.4 MB/s  0:00:00
  Using cached pymorphy3-2.0.4-py3-none-any.whl.metadata (2.4 kB)
  Using cached dawg2_python-0.9.0-py3-none-any.whl.metadata (7.5 kB)
  Using cached pymorphy3_dicts_ru-2.4.417150.4580142-py2.py3-none-any.whl.metadata (2.0 kB)
Using cached pymorphy3-2.0.4-py3-none-any.whl (54 kB)
Using cached dawg2_python-0.9.0-py3-none-any.whl (9.3 kB)
Using cached pymorphy3_dicts_ru-2.4.417150.4580142-py2.py3-none-any.whl (8.4 MB)

   -------------------- ------------------- 2/4 [pymorphy3]
   ------------------------------ --------- 3/4 [ru-core-news-sm]
   ---------------------------------------- 4/4 [ru-core-news-sm]

✔ Download and installation successful
You can now load the package via spac

In [6]:
nlp = spacy.load("ru_core_news_sm")

In [7]:
spacy.prefer_gpu() # Используется ли ускорение на gpu

True

## test

In [8]:
text = ("""Используйте трансформерные модели, если нужна максимальная нагрузка на GPU
Стандартные маленькие модели (_sm) часто не используют GPU по полной, для максимума берите _trf версии 
(например, ru_core_news_trf), которые построены на PyTorch и используют CUDA для ускорения.
""")

In [9]:
doc = nlp(text)

In [10]:
# Analyze syntax
print("Verbs:", [token.lemma_ for token in doc if token.pos_ == "VERB"])

Verbs: ['использовать', 'использовать', 'брать', 'построить', 'использовать']


# Data

## Load

In [11]:
import pandas as pd

In [12]:
path_to_train = r"data\train.csv"
df = pd.read_csv(path_to_train, sep=';')

## Convert

In [13]:
# Конвертация датасета csv в spacy формат
import ast
from spacy.tokens import DocBin

output_path = "data/train.spacy"

db = DocBin()

for (text, annotations_str) in df.to_numpy():
    # перевод "python code" -> python code
    annotations = ast.literal_eval(annotations_str)

    end_text = len(text)

    doc = nlp.make_doc(text)
    ents = []
    for start, end, label in annotations:
        # Нужно изменить alignment_mode, тк есть опечатки в датасете + небольшие корректировки разметки
        end = end_text if end > end_text else end
        start = 0 if start < 0 else start
        span = doc.char_span(start, end, label=label, alignment_mode="expand")
        ents.append(span)

    doc.ents = ents
    db.add(doc)

db.to_disk(output_path)

## Пример считывания из spacy файлы

In [14]:
from spacy.tokens import DocBin

In [15]:
# Загружаем данные из файла
with open(r"data\train.spacy", "rb") as f:
    doc_bin_bytes = f.read()

In [16]:
db = DocBin().from_bytes(doc_bin_bytes)

In [17]:
docs = list(db.get_docs(nlp.vocab))

In [18]:
for doc in docs:
    if doc.text == "стиральный  порош":
        print(doc.text)
        # Можно получить сущности, токены и др.
        for ent in doc.ents:
            print(ent.text, ent.label_)

стиральный  порош
стиральный   B-TYPE
порош I-TYPE
